# 1 Colors_1 classification model

In [95]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

### Define image and batch size

In [96]:
IMAGE_SIZE = 90
BATCH_SIZE = 32

### Import train, test, and val

In [97]:
colors_1_train = utils.image_dataset_from_directory("datasets/data/split/colors_1/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_1_val = utils.image_dataset_from_directory("datasets/data/split/colors_1/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_1_test = utils.image_dataset_from_directory("datasets/data/split/colors_1/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 7267 files belonging to 15 classes.
Found 1550 files belonging to 15 classes.
Found 1556 files belonging to 15 classes.


### Augmentation

In [98]:
colors_1_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

### Define Model

In [99]:
colors_1_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = colors_1_data_augmentation(colors_1_inputs)

x = layers.Rescaling(1./255)(colors_1_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.GlobalAveragePooling2D()(x)

x = layers.Flatten()(x)

colors_1_outputs = layers.Dense(15, activation="softmax")(x)

colors_1_model = keras.Model(inputs=colors_1_inputs, outputs=colors_1_outputs)

colors_1_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

colors_1_model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 90, 90, 3)]       0         
                                                                 
 rescaling_14 (Rescaling)    (None, 90, 90, 3)         0         
                                                                 
 conv2d_66 (Conv2D)          (None, 88, 88, 32)        896       
                                                                 
 max_pooling2d_52 (MaxPoolin  (None, 44, 44, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_67 (Conv2D)          (None, 42, 42, 64)        18496     
                                                                 
 max_pooling2d_53 (MaxPoolin  (None, 21, 21, 64)       0         
 g2D)                                                     

In [100]:
colors_1_callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/2_firstAugmentation_colors_1_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

### Run Model

In [101]:
history = colors_1_model.fit(
  colors_1_train.cache(),
  epochs=30,
  callbacks=colors_1_callbacks_list,
  validation_data=colors_1_val.cache()
)

Epoch 1/30


2022-07-30 06:49:53.815871: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


228/228 [==============================] - ETA: 0s - loss: 1.6006 - accuracy: 0.4796

2022-07-30 06:50:04.094131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


228/228 [==============================] - 12s 49ms/step - loss: 1.6006 - accuracy: 0.4796 - val_loss: 1.2941 - val_accuracy: 0.5258
Epoch 2/30
228/228 [==============================] - 10s 43ms/step - loss: 0.9805 - accuracy: 0.6686 - val_loss: 1.3233 - val_accuracy: 0.6161
Epoch 3/30
228/228 [==============================] - 10s 43ms/step - loss: 0.7942 - accuracy: 0.7290 - val_loss: 1.1869 - val_accuracy: 0.5987
Epoch 4/30
228/228 [==============================] - 9s 41ms/step - loss: 0.6724 - accuracy: 0.7761 - val_loss: 0.8606 - val_accuracy: 0.7071
Epoch 5/30
228/228 [==============================] - 9s 41ms/step - loss: 0.6021 - accuracy: 0.8078 - val_loss: 0.8984 - val_accuracy: 0.7245
Epoch 6/30
228/228 [==============================] - 10s 43ms/step - loss: 0.5448 - accuracy: 0.8254 - val_loss: 1.4706 - val_accuracy: 0.5710
Epoch 7/30
228/228 [==============================] - 10s 45ms/step - loss: 0.4692 - accuracy: 0.8514 - val_loss: 1.2200 - val_accuracy: 0.6394
Epoch

In [110]:
# colors_1_model.evaluate(colors_1_test)

colors_1_model_eval = keras.models.load_model("models/2_firstAugmentation_colors_1_checkpoint_path.keras")
colors_1_model_eval.evaluate(colors_1_test)

2022-07-30 07:00:48.268620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


49/49 [==============================] - 1s 17ms/step - loss: 0.9264 - accuracy: 0.7943


[0.9263811111450195, 0.7943444848060608]

# 2 Vehicle Type classification

### Import train, test, and val datasets

In [103]:
vehicle_type_train = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/train/", image_size=(224, 224), batch_size=16)
vehicle_type_val = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/val", image_size=(224, 224), batch_size=16)
vehicle_type_test = utils.image_dataset_from_directory("datasets/data/split/vehicle_type/test", image_size=(224, 224), batch_size=16)

Found 840 files belonging to 3 classes.
Found 180 files belonging to 3 classes.
Found 180 files belonging to 3 classes.


### Augmentation

In [104]:
vehicle_type_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

### Define Model

In [105]:
vehicle_type_inputs = keras.Input(shape=(224, 224, 3))

x = vehicle_type_data_augmentation(vehicle_type_inputs)

x = layers.Rescaling(1./255)(vehicle_type_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu")(x)

# x = layers.Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)

vehicle_type_outputs = layers.Dense(3, activation="softmax")(x)

vehicle_type_model = keras.Model(inputs=vehicle_type_inputs, outputs=vehicle_type_outputs)

vehicle_type_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

vehicle_type_model.summary()

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_15 (Rescaling)    (None, 224, 224, 3)       0         
                                                                 
 conv2d_70 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_71 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                     

### Callbacks

In [106]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/3_secondAugmentation_vehicle_type_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

### Run the Model

In [107]:
history = vehicle_type_model.fit(
  vehicle_type_train.cache(),
  epochs=30,
  callbacks=callbacks_list,
  validation_data=vehicle_type_val.cache()
)

Epoch 1/30


2022-07-30 06:54:36.170997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - ETA: 0s - loss: 1.2319 - accuracy: 0.3417

2022-07-30 06:54:45.914341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 12s 201ms/step - loss: 1.2319 - accuracy: 0.3417 - val_loss: 1.0936 - val_accuracy: 0.3333
Epoch 2/30
53/53 [==============================] - 11s 202ms/step - loss: 1.1042 - accuracy: 0.3810 - val_loss: 1.0213 - val_accuracy: 0.5500
Epoch 3/30
53/53 [==============================] - 11s 209ms/step - loss: 1.2447 - accuracy: 0.5155 - val_loss: 0.8898 - val_accuracy: 0.5833
Epoch 4/30
53/53 [==============================] - 11s 211ms/step - loss: 1.0009 - accuracy: 0.5500 - val_loss: 0.8526 - val_accuracy: 0.6333
Epoch 5/30
53/53 [==============================] - 11s 209ms/step - loss: 0.9318 - accuracy: 0.5940 - val_loss: 0.8191 - val_accuracy: 0.6111
Epoch 6/30
53/53 [==============================] - 11s 203ms/step - loss: 0.8678 - accuracy: 0.6393 - val_loss: 0.9330 - val_accuracy: 0.5611
Epoch 7/30
53/53 [==============================] - 11s 209ms/step - loss: 0.7570 - accuracy: 0.6964 - val_loss: 0.7195 - val_accuracy: 0.7056
Epoch 8/30

In [109]:
vehicle_types_eval = keras.models.load_model("models/3_secondAugmentation_vehicle_type_checkpoint_path.keras")
vehicle_types_eval.evaluate(vehicle_type_test)
# vehicle_type_model.evaluate(vehicle_type_test)

2022-07-30 06:59:21.169610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 1s 40ms/step - loss: 0.3525 - accuracy: 0.8778


[0.3525424301624298, 0.8777778148651123]

# 3 Colors_2 dataset

In [135]:
IMAGE_SIZE = 90
BATCH_SIZE = 32

In [136]:
colors_2_train = utils.image_dataset_from_directory("datasets/data/split/colors_2/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_2_val = utils.image_dataset_from_directory("datasets/data/split/colors_2/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
colors_2_test = utils.image_dataset_from_directory("datasets/data/split/colors_2/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 10917 files belonging to 8 classes.
Found 2337 files belonging to 8 classes.
Found 2347 files belonging to 8 classes.


### Augmentation

In [137]:
colors_2_data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
] 
)

In [138]:
colors_2_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

x = colors_2_data_augmentation(colors_2_inputs)

x = layers.Rescaling(1./255)(colors_2_inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x) 
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x) 
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.GlobalAveragePooling2D()(x)

x = layers.Flatten()(x)

colors_2_outputs = layers.Dense(8, activation="softmax")(x)

colors_2_model = keras.Model(inputs=colors_2_inputs, outputs=colors_2_outputs)

colors_2_model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

colors_2_model.summary()

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 90, 90, 3)]       0         
                                                                 
 rescaling_25 (Rescaling)    (None, 90, 90, 3)         0         
                                                                 
 conv2d_111 (Conv2D)         (None, 88, 88, 32)        896       
                                                                 
 max_pooling2d_86 (MaxPoolin  (None, 44, 44, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_112 (Conv2D)         (None, 42, 42, 64)        18496     
                                                                 
 max_pooling2d_87 (MaxPoolin  (None, 21, 21, 64)       0         
 g2D)                                                     

### Callbacks

In [139]:
colors_2_callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/1_base_colors_1_checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

In [140]:
history = colors_2_model.fit(
  colors_2_train.cache(),
  epochs=30,
  callbacks=colors_2_callbacks_list,
  validation_data=colors_2_val.cache()
)

Epoch 1/30


2022-07-30 07:22:14.914074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


342/342 [==============================] - ETA: 0s - loss: 0.9532 - accuracy: 0.6761

2022-07-30 07:22:30.235878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


342/342 [==============================] - 17s 48ms/step - loss: 0.9532 - accuracy: 0.6761 - val_loss: 0.7938 - val_accuracy: 0.7082
Epoch 2/30
342/342 [==============================] - 14s 41ms/step - loss: 0.4671 - accuracy: 0.8400 - val_loss: 0.3817 - val_accuracy: 0.8695
Epoch 3/30
342/342 [==============================] - 15s 43ms/step - loss: 0.3435 - accuracy: 0.8806 - val_loss: 0.3045 - val_accuracy: 0.8909
Epoch 4/30
342/342 [==============================] - 15s 44ms/step - loss: 0.2732 - accuracy: 0.9071 - val_loss: 0.3213 - val_accuracy: 0.8926
Epoch 5/30
342/342 [==============================] - 16s 47ms/step - loss: 0.2316 - accuracy: 0.9235 - val_loss: 0.2479 - val_accuracy: 0.9157
Epoch 6/30
342/342 [==============================] - 16s 46ms/step - loss: 0.1966 - accuracy: 0.9363 - val_loss: 0.2807 - val_accuracy: 0.9123
Epoch 7/30
342/342 [==============================] - 16s 47ms/step - loss: 0.1711 - accuracy: 0.9427 - val_loss: 0.2761 - val_accuracy: 0.9089
Epo

In [141]:
# colors_2_model.evaluate(colors_2_test)

colors_2_eval = keras.models.load_model("models/1_base_colors_1_checkpoint_path.keras")
colors_2_eval.evaluate(colors_2_test)


2022-07-30 07:25:25.924046: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


74/74 [==============================] - 2s 21ms/step - loss: 0.3316 - accuracy: 0.8922


[0.3316270709037781, 0.8922027945518494]